# Assignment: Week 2 - Goal search

## Getting Git
Throughout this course we will make use of the [aima-python project on Github](https://github.com/aimacode/aima-python).  This is an active opensource development effort with many contributors including atleast one of the text authors. 

To get the code you will need to get a copy of it.  This is commonly referred to as "cloning" a repository and you can find directions how to do it [here](https://help.github.com/articles/cloning-a-repository/).  Git is what is called version control software and is used to power the web-site GitHub.  You can interact with GitHub simply through your web browser, or by using client software.  I recommend sooner but the choice is up to you.  If you are using Linux or macOS, then you probably already have Git installed.  Enter `which git` at the command line, you will either see a path or `command not found`.  These lists of clients will provide you plenty of options to choose from:

* [Command-line clients](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git)
* [GUI clients](https://git-scm.com/downloads/guis)

**If you are new to Git -- that is awesome -- you will learn some Git basics that will help prepare you for working as a data scientist.  You do not need to know Git or anything but a novice user in this course.**   

## Cloing the repo
To get a copy of all the code in the repository, you need to `clone` it.  This simply means "copy it" and allows you to interact with others in the development of open source software.  The aima-python repository can be found at [github.com/aimacode/aima-python](https://github.com/aimacode/aima-python).  To clone simply enter `git clone https://github.com/aimacode/aima-python` from the command line.  Git will create a `aima-python` folder in whatever directory you ran the command from.  If you are using a GUI, find clone and provide it with `https://github.com/aimacode/aima-python`.   Once the command runs you should have the code downloaded.  Don't worry, you can delete this folder and repeat the process again and again.  

## Preparing to work on this assignment

To complete this assignment here are the steps you will need to complete: 

1. Download and copy this assignment into your repository

2. Start Jupyter from above it and navigate into it (pretty much your only option in Windows)

3. Start Jupyter in that directory 

## Import the needed packages

To do this you will need to be running your Jupyter notebook from the aima-python directory.  This is where your aima-python repository is cloned.  If you get errors when you go to import the python packages needed for this assignment, then you are probably have not specified the path correctly.  There are two ways to check:
1. `%pwd` - The present working directory magic shows in what directory Jupyter's is currentry running.
2. `%ls` - The list magic shows the files in Jupyter's current working directory.


From the command line you can start the notebook from wherever you like, if you use Anaconda Navigator, then it starts in location specific to your OS.  

In [2]:
%pwd

'/Users/rob/Documents/GitHub/ai-course/assignments'

Below is where you change directories to where the aima-python code is located

In [3]:
%cd ~/Documents/Github/aima-python

/Users/rob/Documents/GitHub/aima-python


In [4]:
%pwd

'/Users/rob/Documents/GitHub/aima-python'

In [5]:
from search import *
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

In [6]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

## Define the problem class

A problem is something to solve.  Here we represent it as python class.  Run the following cell and read the brief documentation of its methods. 

psource(Problem)

## Elements of the graph

Graphs are built from a network of interconnected nodes.  Each node has a link to other nodes.  This is an object-oriented approach to representing a graph.  Read over the descriptions of what each method does.  

TODO - Determine why this notebook is being rendered differently than search.ipynb

In [7]:
# psource(Node)

In [8]:
# psource(GraphProblem)

## Representing a map as a graph

To illustrate how the examples in our textbook, we too will use a map of Romania. With apologies to Romainians everywhere, I will abbreviate the city names to one letter.  Compare the map and the data structure below.  Notice that from A you can get to Z, S and T and the distances between these nodes.  

![Figure 3.2 Road map of Romania](../images/Figure-S3-2.png)

In [9]:
# Distances from a node to one or more nodes.  
romania_map = UndirectedGraph(dict(
    A=dict(Z=75, S=140, T=118),
    B=dict(U=85, P=101, G=90, F=211),
    C=dict(D=120, R=146, P=138),
    D=dict(M=75),
    E=dict(H=86),
    F=dict(S=99),
    H=dict(U=98),
    I=dict(V=92, N=87),
    L=dict(T=111, M=70),
    O=dict(Z=71, S=151),
    P=dict(R=97),
    R=dict(S=80),
    U=dict(V=142)))

romania_map.locations = dict(
    A=(91, 492), B=(400, 327), C=(253, 288),
    D=(165, 299), E=(562, 293), F=(305, 449),
    G=(375, 270), H=(534, 350), I=(473, 506),
    L=(165, 379), M=(168, 339), N=(406, 537),
    O=(131, 571), P=(320, 368), R=(233, 410),
    S=(207, 457), T=(94, 410), U=(456, 350),
    V=(509, 444), Z=(108, 531))

Use python's `type(object)` operator to find out what type of the romania_map object

In [10]:
type(romania_map)

search.Graph

Now find out what methods is offers by typing `romanian_map.` and then press the _TAB_ key.  You should see a drop down box with two methods: `connect` and `connect1`.  

In [ ]:
romania_map.connect?

Now follow let's get the documentation for these functions so we can figure out how to use them.  To do this, enter `romania_map.connect?` then press _SHIFT + TAB_ simultaneously.  

In [ ]:
romania_map

## Create an instance of it

In [ ]:
my_problem = HelloProblem(initial_state='')

## Solve the problem
Including a viewer is handy to see how the algorithm goes about solving the problem.  For large search spaces this can take a very long time, so small examples and or efficeint algorithms are recommended. 

In [ ]:
from simpleai.search.viewers import WebViewer

In [ ]:
result = astar(my_problem, viewer=WebViewer())

## What to expect

```
Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)`

Starting the WebViewer, access it from your web browser, navigating to the address:
http://localhost:8000
To stop the WebViewer, use the "Stop running" link (on the viewer site, from the browser)

127.0.0.1 - - [19/May/2018 15:42:21] "GET / HTTP/1.1" 200 -
...
```

## Visualizing the search

Open the `http://0.0.0.0:8000` page in your browser to see the visualization.  Then, get an understanding of the visualation by reading through brief documentation.  Next, start driving the web application forward using the menu in the upper left corner.  